In [26]:
import pandas as pd
raw = pd.read_csv('https://raw.githubusercontent.com/automoto/big-five-data/master/big_five_scores.csv', ',')
# this data contains the results from over 300,000 induviduals so it may take a few seconds.

In [27]:
raw.head(5) # Let's see how the data looks like

,case_id,country,age,sex,agreeable_score,extraversion_score,openness_score,conscientiousness_score,neuroticism_score
0,1,South Afri,24,1,0.753333,0.496667,0.803333,0.886667,0.426667
1,3,UK,24,2,0.733333,0.680000,0.786667,0.746667,0.590000
2,4,USA,36,2,0.880000,0.770000,0.860000,0.896667,0.296667
3,5,UK,19,1,0.690000,0.616667,0.716667,0.636667,0.563333
4,6,UK,17,1,0.600000,0.713333,0.646667,0.633333,0.513333


In [28]:
raw = raw.drop('age', axis=1) # droping the age column for now, but the country column might be relevant

In [29]:
countries = pd.DataFrame(raw['country'].value_counts())
countries.median() # there are alot of countries with very few subjects

country    26.0
dtype: float64

In [30]:
countries = countries.loc[countries['country'] >= 200]
countries.median()

country    638.5
dtype: float64

In [31]:
len(countries) # so we're left with 48 countries

48

In [34]:
raw = raw.loc[raw['country'].isin(countries.index)] # keep only those countries

In [36]:
len(raw['country'].value_counts()) # seems like it worked!

48

Now the problem is that most skl algorithms dont know how to use strings for their predictions. So, we will split
the column 'country' into 48 columns of countries which will contain ones and zeros:
one if the subjecct is from the column's name (country)
or zero if not.
in each row there sould be exactly one "1".

In [39]:
raw['country'].str.get_dummies()

,Afghanista,Albania,Algeria,Argentina,Australia,Austria,Belgium,Brazil,Canada,China,...,South Afri,South Kore,Spain,Sweden,Switzerlan,Taiwan,Thailand,Turkey,UK,USA
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307307,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
307308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
307309,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
307310,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [56]:
n_data = raw.join(raw['country'].str.get_dummies(), how='right') # combining the two data frames into a n_data

In [60]:
n_data.head(5)

,case_id,sex,agreeable_score,extraversion_score,openness_score,conscientiousness_score,neuroticism_score,Afghanista,Albania,Algeria,...,South Afri,South Kore,Spain,Sweden,Switzerlan,Taiwan,Thailand,Turkey,UK,USA
0,1,1,0.753333,0.496667,0.803333,0.886667,0.426667,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,3,2,0.733333,0.680000,0.786667,0.746667,0.590000,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,4,2,0.880000,0.770000,0.860000,0.896667,0.296667,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,5,1,0.690000,0.616667,0.716667,0.636667,0.563333,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,6,1,0.600000,0.713333,0.646667,0.633333,0.513333,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [59]:
n_data = n_data.drop('country', axis=1) # we ahve no use for the 'country' column now

In [61]:
n_data.head(5) 

,case_id,sex,agreeable_score,extraversion_score,openness_score,conscientiousness_score,neuroticism_score,Afghanista,Albania,Algeria,...,South Afri,South Kore,Spain,Sweden,Switzerlan,Taiwan,Thailand,Turkey,UK,USA
0,1,1,0.753333,0.496667,0.803333,0.886667,0.426667,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,3,2,0.733333,0.680000,0.786667,0.746667,0.590000,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,4,2,0.880000,0.770000,0.860000,0.896667,0.296667,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,5,1,0.690000,0.616667,0.716667,0.636667,0.563333,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,6,1,0.600000,0.713333,0.646667,0.633333,0.513333,0,0,0,...,0,0,0,0,0,0,0,0,1,0
